In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from eden.util import configure_logging
import logging
configure_logging(logging.getLogger(),verbosity=2)
%matplotlib inline
!pwd
import random
random.seed(31337)

In [ ]:
'''
somehow we need to get RNA data...
'''
from eden.converter.fasta import fasta_to_sequence
from eden.converter.rna.rnafold import rnafold_to_eden
import graphlearn.abstract_graphs.rnasampler as rspl
from eden.graph import Vectorizer
def rfam_uri(family_id):
    return 'http://rfam.xfam.org/family/%s/alignment?acc=%s&format=fastau&download=0'%(family_id,family_id)

def rfam_uri(family_id):
    return '%s.fa'%(family_id)


def get_graphs(rfam_id = 'RF00005'):
    seqs = fasta_to_sequence(rfam_uri(rfam_id))
    graphs = rnafold_to_eden(seqs, shape_type=5, energy_range=30, max_num=3)
    return graphs


vectorizer=Vectorizer()

def get_directed_graphs(rfam_id = 'RF00005'):
    for g in get_graphs(rfam_id):
        g=vectorizer._edge_to_vertex_transform(g)
        yield rspl.expanded_rna_graph_to_digraph(g)



In [ ]:

graphs=get_directed_graphs()



In [ ]:
g=graphs.next()
from graphlearn.utils import draw

# note: show direction doesnt work?
draw.draw_center(g,root_node=0,radius=99,show_direction=True,contract=False, size = 10,vertex_label='id')


import graphlearn.abstract_graphs.rnasampler as rspl
g= rspl.direct_abstractor(g,4)

from graphlearn.utils import draw
for n,d in g.nodes(data=True):
    d['contr']= str(d.get('contracted', ''))                    
draw.graphlearn_draw(g, size = 10,vertex_label = 'contr')



In [ ]:
'''
1. MAKE SURE THAT WE CAN EXTRACT DIRECTED CIPS
'''
%matplotlib inline
from eden.converter.graph.gspan import gspan_to_eden
from graphlearn import graphtools as gt
import graphlearn.utils.draw as myutils
import eden.graph as eg
import graphlearn.graphlearn as gl
import numpy as np
#from eden.util.display import graphlearn_draw

#get a graph
graphs = get_directed_graphs()
graph=graphs.next()


def mark_and_draw(cip):
    graph=myutils.cip_to_drawable_graph(cip)
    myutils.draw_graph_set_graphlearn(graph, 
                           n_graphs_per_line=7, size=5, 
                           colormap='Paired', invert_colormap=False,node_border=1,
                           vertex_alpha=0.6, edge_alpha=0.2, node_size=450)
    
radius_list=[0,2]
thickness_list=[2,4]
cip=gl.extract_core_and_interface(1,graph,radius_list=radius_list,thickness_list=thickness_list)
#mark_and_draw(cip)
myutils.graphlearn_draw(  cip[0].graph, contract=False)
myutils.graphlearn_draw(cip[0].graph,edge_label=None)

In [ ]:
%%time
'''
getting the grammar is the same as in the UberSampler so there is nothing interesting here..
if you need to debug do so in the UberSampler
'''
import os 
os.nice(20)
%matplotlib inline
import graphlearn.utils.draw as myutils
import graphlearn.abstract_graphs.rnasampler as rspl
import itertools
import graphlearn.localsubstitutablegraphgrammar as grammar


# learn a grammar
graphs = get_directed_graphs()
graphs = itertools.islice(graphs,100)


sampler=rspl.RNASampler(radius_list=[0,1],thickness_list=[1],base_thickness_list=[.5], min_cip_count=1, min_interface_count=2)
sampler.fit(graphs,n_jobs=4,batch_size=1)
sampler.save('tmp/rna_ubergrammar.ge')

In [ ]:
#draw production rules
myutils.draw_grammar(sampler.lsgg.productions,n_productions=5,n_graphs_per_production=10,
                     n_graphs_per_line=6, size=10, contract=False,
                     colormap='Paired', invert_colormap=False,node_border=1,
                     vertex_alpha=0.6, edge_alpha=0.5, node_size=250)

    

In [ ]:
%%time
'''
RNA sampling
'''
import os
os.nice(19)

#import mpld3
#mpld3.enable_notebook()
import graphlearn.utils.draw as myutils
import graphlearn.abstract_graphs.rnasampler as rspl
from eden.converter.graph.gspan import gspan_to_eden
import itertools
import matplotlib.pyplot as plt
import graphlearn.utils.draw as myutils
import eden.graph as ge

sampler=rspl.RNASampler()
sampler.load('tmp/rna_ubergrammar.ge')
graphs = get_graphs()

id_start=15
id_end=id_start+9
graphs = itertools.islice(graphs,id_start,id_end)
n_steps=50


def rm_bonds(graphs):
    for graph in graphs:
        graph=graph.copy()
        bonds= [ (a,b) for a,b,d in graph.edges(data=True) if d['label']== '=' ]
        graph.remove_edges_from(bonds)
        yield graph

graphs = sampler.sample(graphs,
                        n_samples=7,
                        batch_size=1,
                        n_steps=n_steps,
                        n_jobs=1,
                        quick_skip_orig_cip=True,
                        probabilistic_core_choice=True,
                        burnin=0,
                        improving_threshold=0.5,
                        select_cip_max_tries=200,
                        keep_duplicates=True)

from eden.modifier.graph.vertex_attributes import colorize   
scores=[]
ids=range(id_start,id_end)
for i,graph in enumerate(graphs):
    print 'Graph id: %d'%(ids[i])
    scores.append(graph.graph['sampling_info']['score_history'])
    path_graphs = graph.graph['sampling_info']['graphs_history']
    
    path_graphs = colorize(path_graphs,
                           output_attribute = 'color_level', 
                           labels = ['A','U','G','C'])
    path_graphs= list(path_graphs)
    #myutils.draw_graph_set(rm_bonds(path_graphs), size=10)
    myutils.graphlearn_draw(path_graphs,
                           n_graphs_per_line=3, size=20, 
                           colormap='Paired', invert_colormap=False,node_border=0.5, vertex_color='color_level',
                           vertex_alpha=0.5, edge_alpha=0.7, node_size=450,edge_label='label',
                          headlinehook= myutils.get_score_of_graph
                          )
    

In [ ]:
%matplotlib inline
from itertools import islice
import numpy as np

step=1
num_graphs_per_plot=3
num_plots=np.ceil([len(scores)/num_graphs_per_plot])
for i in range(num_plots):
    plt.figure(figsize=(10,5))
    for j,score in enumerate(scores[i*num_graphs_per_plot:i*num_graphs_per_plot+num_graphs_per_plot]):
        data = list(islice(score,None, None, step))
        plt.plot(data, label='graph %d'%(j+i*num_graphs_per_plot+id_start))
    plt.legend(loc='lower right')
    plt.grid()
    plt.ylim(-0.1,1.1)
    plt.show()

In [ ]:

import networkx as nx
import graphlearn.utils.draw as draw
from eden.graph import Vectorizer

v=Vectorizer()
def getpathgraph(labels):
    nlabels= len(labels)
    G=nx.path_graph(nlabels)
    for e in range(nlabels):
        G.node[e]['label']=labels[e]
    for e in range(nlabels-1):
        G.edge[e][e+1]['label']='.'
    return G

g=getpathgraph("ABC")

g= v._edge_to_vertex_transform(g)

draw.graphlearn_draw(g,contract=False,show_direction=True)

g2=nx.DiGraph(g)
#draw.display(g2,contract=False,vertex_label='id')

for n,d in g2.nodes(data=True):
    if 'edge' in d:
        if d['label']=='.':
            ns=g2.neighbors(n)
            g2.remove_edge(ns[1],n)
            g2.remove_edge(n,ns[0])
          
       
#g2 = g2.to_undirected()
draw.graphlearn_draw(g2,contract=False,vertex_label='id',show_direction=True)
g2.edges()
for n in g2.nodes():
    print n,g2.neighbors(n)
print isinstance(g2, nx.DiGraph) 

In [ ]:
g2.edges(0,data=True)

In [ ]:
asd = g2.subgraph(range(2)).copy()

In [ ]:
class A(object):
    def __init__(self,a=123):
        self.a=a
        
        
class B(A):
    def __init__(self,b=122):
        super(B,self).__init__(b)
        
        

class C(A):
    def __init__(self,c=121):
        super(C,self).__init__(c)
        
        
c=C()
c.a
        
        
        
    
    